In [22]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
import os  # For path checking
import time  # For introducing delays

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=0)
mp_drawing = mp.solutions.drawing_utils

# Initialize text-to-speech engine globally
engine = pyttsx3.init()

def give_audio_feedback(feedback_list):
    try:
        if feedback_list:
            for feedback in feedback_list:
                engine.say(feedback)
        else:
            engine.say("Great! Your pose looks correct.")
        engine.runAndWait()
    except RuntimeError as e:
        print("Error during audio feedback:", e)

def extract_landmarks(image_path, pose_model):
    if not os.path.exists(image_path):
        print("Error: File not found:", image_path)
        return None
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose_model.process(image_rgb)
    if results.pose_landmarks:
        selected_landmarks = [0, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
        return np.array([[lm.x, lm.y, lm.z] for i, lm in enumerate(results.pose_landmarks.landmark) if i in selected_landmarks]).flatten()
    else:
        print("No pose detected in reference image.")
        return None

def compare_landmarks(predicted_landmarks, reference_landmarks, frame, tolerance=0.25):
    predicted_landmarks = predicted_landmarks.reshape(-1, 3)
    reference_landmarks = reference_landmarks.reshape(-1, 3)
    deviations = np.linalg.norm(predicted_landmarks - reference_landmarks, axis=1)
    feedback = []
    for i, deviation in enumerate(deviations):
        if deviation > tolerance:
            feedback.append(f"Keypoint {i} is off by {deviation:.2f} units.")
            cv2.putText(frame, f"Keypoint {i} off", (20, 60 + i * 30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
    if not feedback:
        cv2.putText(frame, "Pose is correct!", (10, 40), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "Adjust your pose!", (10, 40), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
    return feedback

def detect_pose(frame, pose_model):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_model.process(image_rgb)
    if results.pose_landmarks:
        selected_landmarks = [0, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
        landmarks = np.array([[lm.x, lm.y, lm.z] for i, lm in enumerate(results.pose_landmarks.landmark) if i in selected_landmarks])
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        return frame, landmarks
    return frame, None

# Load the reference pose landmarks
reference_landmarks = extract_landmarks(r'C:\Users\HP\Downloads\DATASET\DATASET\TRAIN\tree\00000075.jpg', pose_video)
if reference_landmarks is None:
    print("Error: Could not extract reference landmarks.")
    exit()

# Load two images for display
hello_image = cv2.imread(r'C:\Btech Project\Sign language data\collectedimages\hello\hello.de07e05d-ce81-11ef-9c08-80913381312a.jpg')
second_image = cv2.imread(r'C:\Btech Project\Sign language data\collectedimages\no\no.1b1a6092-ce82-11ef-985f-80913381312a.jpg')  # Second image path

if hello_image is None or second_image is None:
    print("Error: One or both images not found.")
    exit()

camera_video = cv2.VideoCapture(0)  # Use 0 for the default camera
if not camera_video.isOpened():
    print("Error: Unable to access camera.")
    exit()

# Variable to control image display timing
image_to_display = "hello"  # Initial image to display
last_image_time = time.time()  # Time when the last image was displayed

while camera_video.isOpened():
    ok, frame = camera_video.read()
    if not ok:
        break

    frame = cv2.flip(frame, 1)

    # Check if 2 seconds have passed to switch the image
    current_time = time.time()
    if current_time - last_image_time >= 2:
        if image_to_display == "hello":
            image_to_display = "second"
            last_image_time = current_time
        else:
            image_to_display = "hello"
            last_image_time = current_time

    # Detect pose
    frame, landmarks = detect_pose(frame, pose_video)

    if landmarks is not None:
        feedback = compare_landmarks(landmarks, reference_landmarks, frame, tolerance=0.25)

        # Resize the appropriate image based on the time
        if image_to_display == "hello":
            hello_resized = cv2.resize(hello_image, (150, 150))  # Adjust size as needed
            frame_h, frame_w, _ = frame.shape
            hello_h, hello_w, _ = hello_resized.shape
            hello_x_offset = frame_w - hello_w - 10  # Bottom-right corner
            hello_y_offset = frame_h - hello_h - 10  # Bottom-right corner
            frame[hello_y_offset:hello_y_offset+hello_h, hello_x_offset:hello_x_offset+hello_w] = hello_resized
        else:
            second_resized = cv2.resize(second_image, (150, 150))  # Adjust size as needed
            frame_h, frame_w, _ = frame.shape
            second_h, second_w, _ = second_resized.shape
            second_x_offset = frame_w - second_w - 10  # Bottom-right corner
            second_y_offset = frame_h - second_h - 10  # Bottom-right corner
            frame[second_y_offset:second_y_offset+second_h, second_x_offset:second_x_offset+second_w] = second_resized

        # Provide audio feedback
        give_audio_feedback(feedback)

    # Display the frame
    cv2.imshow("Yoga Pose Feedback", frame)

    # Exit on pressing the ESC key
    if cv2.waitKey(1) & 0xFF == 27:
        break

camera_video.release()
cv2.destroyAllWindows()


Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: run loop already started
Error during audio feedback: